* https://docs.google.com/document/d/1b7DlLtvrVxihcO65dsBaIjUU-a6aLjdQfNHSOM2PhUM/edit?tab=t.0#heading=h.v0phdvfalpr2

## setup and tryout

In [2]:
%load_ext dotenv
%dotenv

In [6]:
import os

# from openai import OpenAI
import jupyter_black

jupyter_black.load()

In [9]:
%%time
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

response = llm.invoke("What are the best practices for developing with LangChain?")

print(response.content)

Developing with LangChain effectively requires understanding its core components and applying best practices for efficiency, maintainability, and responsible AI usage. Here's a breakdown of key areas and recommended practices:

**1. Understanding LangChain Components & Architecture:**

*   **LLMs (Large Language Models):**  Familiarize yourself with different LLMs (OpenAI, Cohere, Hugging Face models, etc.) and their strengths and weaknesses. Choose the right model for your task (e.g., GPT-3.5 for cost-effectiveness, GPT-4 for advanced reasoning).
*   **Prompts & Prompt Templates:**  Master prompt engineering. Understand how to craft effective prompts that guide the LLM's behavior. Use prompt templates for reusability and parameterization.
*   **Chains:**  Learn how to chain LLMs and other components together. Chains enable complex workflows, from simple summarization to sophisticated question-answering systems.
*   **Indexes:**  Utilize indexes (e.g., document loaders, retrievers, vec

original:

```python
from langchain_openai.chat_models import ChatOpenAI
# chat = ChatOpenAI(openai_api_key="...")
# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
chat = ChatOpenAI()
chat.invoke("Hello, how are you?") 
```

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable.
# You can get one from Google AI Studio: https://aistudio.google.com/app/apikey

# Initialize the Gemini chat model
# You can also specify other models like "gemini-1.5-pro"
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# Invoke the model with a prompt
response = chat.invoke("Hello, how are you?")
print(response.content)

I am doing well, thank you for asking! As a large language model, I don't experience emotions like humans do, but I am functioning properly and ready to assist you. How can I help you today?


## 85. Chat Models -- Coding

In [19]:
from IPython.display import Markdown

response = chat.invoke("What is the capital of France?")
Markdown(response.content)

The capital of France is **Paris**.

In [20]:
response.response_metadata

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.0-flash-lite',
 'safety_ratings': []}

original:

```python
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)
```

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Ensure your GOOGLE_API_KEY environment variable is set
# 1. Initialize the Gemini chat model
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

# 2. Define the message(s) for the model
text = "What would be a good company name for a company that makes colorful socks?"
messages = [
    SystemMessage(content="You are a helpful assistant that generates company names."),
    HumanMessage(content=text),
]

# 3. Invoke the model with the messages
result = chat.invoke(messages)

# 4. Print the AI's response content
print(result.content)

Here are some company name ideas for a colorful sock company, playing on different angles:

**Emphasizing Color & Fun:**

*   Chromatic Threads
*   Rainbow Footwear
*   Socktacular Colors
*   Kaleidoscope Socks
*   Vivid Soles
*   The Color Sock Shop
*   Happy Feet Hues
*   Sock Pop
*   Technicolor Toes

**Highlighting Quality & Comfort:**

*   Cozy Canvas Socks
*   Comfort & Hue
*   The Soft Sole
*   Thread & Thrive
*   Artisan Socks
*   Stride in Style
*   Foot Haven

**Playful & Catchy:**

*   Sock It To 'Em
*   Sole Mates
*   Toe-riffic Socks
*   Sock Monster
*   The Sock Drawer
*   Get Your Sock On
*   Soleful Creations

**Modern & Minimalist:**

*   Hue & Heel
*   The Sock Lab
*   Bold Threads
*   Stride Studio

**Tips for Choosing:**

*   **Check availability:** Make sure the name isn't already in use and that the domain name is available.
*   **Consider your target audience:** Who are you trying to sell to? (e.g., kids, adults, fashion-conscious individuals)
*   **Keep it memor

## 86. Chat Prompt Templates
* https://drive.google.com/file/d/1JoyxZlYfngmXnvrRyo7qqvUoB7qz6il0/view?usp=drive_link

In [25]:
from langchain_core.prompts.chat import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that generates company names"),
        ("human", "{text}"),
    ]
)

result = chat_prompt_template.invoke(
    {
        # "text": "What would be a good company name for a company that makes colorful socks?"
    }
)

# model = Cha(model='gpt-4o-mini')

ai_llm_result = chat.invoke(result)
print(ai_llm_result.content)

KeyError: "Input to ChatPromptTemplate is missing variables {'text'}.  Expected: ['text'] Received: []\nNote: if you intended {text} to be part of the string and not a variable, please escape it with double curly braces like: '{{text}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "